In [60]:
import pandas as pd
import numpy as np 
import csv 
from datetime import datetime
import itertools
import seaborn as sn

df_og = pd.read_csv("dataset_mood_smartphone.csv")
df = pd.read_csv("df_SVM.csv")
# df = pd.read_csv("dataframe_new.csv")


In [65]:
df[df["screen"].isna()]

,no,target,t,activity,appCat.builtin,appCat.communication,appCat.entertainment,appCat.finance,appCat.game,appCat.office,...,appCat.travel,appCat.unknown,appCat.utilities,appCat.weather,call,circumplex.arousal,circumplex.valence,mood,screen,sms
283,71,6,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,3.0,0.000000,0.0,6.500000,NaN,NaN
286,72,5,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,3.0,0.000000,0.0,6.500000,NaN,NaN
289,73,6,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,3.0,0.000000,0.0,6.500000,NaN,NaN
291,73,6,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,9.0,-0.666667,0.0,5.000000,NaN,3.0
292,74,6,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,3.0,0.000000,0.0,6.500000,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3995,999,7,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,1.000000,1.0,7.000000,NaN,NaN
3996,1000,7,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,1.000000,1.0,7.333333,NaN,NaN
3997,1000,7,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0.400000,1.0,7.000000,NaN,NaN
3998,1000,7,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,1.000000,1.0,7.000000,NaN,NaN


In [62]:
df.isna().sum()
# sn.heatmap(df.corr())
# df.corr("kendall")["target"]

no                         0
target                     0
t                          0
activity                 288
appCat.builtin           271
appCat.communication     313
appCat.entertainment    1372
appCat.finance          3459
appCat.game             3557
appCat.office           3267
appCat.other             460
appCat.social            909
appCat.travel           2779
appCat.unknown          3323
appCat.utilities        2701
appCat.weather          3817
call                    1677
circumplex.arousal         0
circumplex.valence         4
mood                       0
screen                   249
sms                     2733
dtype: int64

In [47]:
for v in df.columns[3:]: 
    for t in range(4): 
        corr = df[df["t"]==t].corr()["target"][v]
        print("var: ", v, "t:", t, "corr:", corr)
        
print(df[df["t"]==3].corr()["target"])
print(df[df["t"]==2].corr()["target"])
print(df[df["t"]==1].corr()["target"])
print(df[df["t"]==0].corr()["target"])


var:  activity t: 0 corr: 0.014212174927915586
var:  activity t: 1 corr: 0.06664695112906653
var:  activity t: 2 corr: 0.07469579302321304
var:  activity t: 3 corr: 0.08282884916876666
var:  appCat.builtin t: 0 corr: -0.01966430670499124
var:  appCat.builtin t: 1 corr: 0.03282937605273743
var:  appCat.builtin t: 2 corr: 0.02173487033160107
var:  appCat.builtin t: 3 corr: 0.04581745059988459
var:  appCat.communication t: 0 corr: 0.107169839560851
var:  appCat.communication t: 1 corr: 0.09798875407046731
var:  appCat.communication t: 2 corr: 0.11618905211962247
var:  appCat.communication t: 3 corr: 0.09298993730329169
var:  appCat.entertainment t: 0 corr: 0.15407890622274611
var:  appCat.entertainment t: 1 corr: 0.13898813925209946
var:  appCat.entertainment t: 2 corr: 0.07579713078038894
var:  appCat.entertainment t: 3 corr: 0.1458167151128887
var:  appCat.finance t: 0 corr: 0.013614313688690736
var:  appCat.finance t: 1 corr: -0.048907510479515304
var:  appCat.finance t: 2 corr: -0.002

In [14]:
def create_daily_df(df): 
    """
    This function aggregates the variable values per day, where it takes the average of mood, arousal and valence. 
    It takes the sum of the other variables. 
    """

    # Add date column to dataframe 
    df["date"] = df["time"].str[:10]

    # Define the variables of which we want the mean per day - all other variables are aggregated in a sum. 
    mean_vars = ["mood", "circumplex.arousal", "circumplex.valence"]

    # Create the dataframe with the means of variables per day  
    df_means = df[df["variable"].isin(mean_vars)]
    df_means =  df_means.groupby(["id", "date", "variable"], as_index=False)[["value"]].mean()

    # Create the datafram with the sums of variables per day
    df_sums = df[~df["variable"].isin(mean_vars)]
    df_sums = df_sums.groupby(["id", "date", "variable"], as_index=False)[["value"]].sum()

    # Concatenate the two dataframes into the df with daily values
    df_daily = pd.concat([df_sums, df_means])
    df_daily = df_daily.sort_values(by=["id", "date"])

    return df_daily

df_daily = create_daily_df(df_og)

In [30]:
len(df_daily["date"].unique())

113

In [29]:
len(df_daily[df_daily["variable"]=="mood"]["date"].unique())

98